In [23]:
%pip install yfinance
%pip install requests_cache
%pip install requests_ratelimiter
%pip install pyrate_limiter
%pip install pyarrow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/25.2 MB ? eta -:--:--
   ------------------------- -------------- 16.0/25.2 MB 84.5 MB/s eta 0:00:01
   ---------------------------------------- 25.2/25.2 MB 77.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import yfinance as yf  # type: ignore

In [3]:
data = yf.download(['MSFT', 'AAPL', 'GOOG'], period='1mo')
changes = data.Close.pct_change()
mean = changes.mean()
# ddof of 1 because its a sample of a population
sd = changes.std(ddof=1).dropna()
print("changes")
print(changes.tail())

print("mean")
print(mean)

print("standard deviation")
print(sd)

[*********************100%***********************]  3 of 3 completed

changes
Ticker          AAPL      GOOG      MSFT
Date                                    
2025-02-05 -0.001418 -0.069376  0.002231
2025-02-06  0.003226  0.000052  0.006122
2025-02-07 -0.023969 -0.031918 -0.014598
2025-02-10  0.001187  0.005664  0.006028
2025-02-11  0.021832 -0.006004 -0.001892
mean
Ticker
AAPL   -0.000118
GOOG   -0.001087
MSFT   -0.000487
dtype: float64
standard deviation
Ticker
AAPL    0.020876
GOOG    0.024378
MSFT    0.020755
dtype: float64


In [4]:
z_scores = (changes - mean) / sd
z_scores

Ticker,AAPL,GOOG,MSFT
Date,,,
2025-01-13,NaN,NaN,NaN
2025-01-14,-0.223211,-0.219940,-0.152097
2025-01-15,0.948167,1.317834,1.256762
2025-01-16,-1.929543,-0.490613,-0.172079
2025-01-17,0.366616,0.707133,0.528438
2025-01-21,-1.523132,0.476495,-0.036073
2025-01-22,0.261697,0.126779,2.013682
2025-01-23,-0.030709,-0.047697,0.078516
2025-01-24,-0.182802,0.521429,-0.262378


In [5]:
bad_data = z_scores[z_scores > 2]
bad_data.dropna(how="all")

Ticker,AAPL,GOOG,MSFT
Date,,,
2025-01-22,NaN,NaN,2.013682


In [6]:
Q1 = changes.quantile(0.10)
Q3 = changes.quantile(0.90)
IQR = Q3 - Q1

# Define outliers as values outside 1.5 * IQR from Q1 or Q3
outliers = (changes < (Q1 - 1.5 * IQR)) | (changes > (Q3 + 1.5 * IQR))
print("Outliers (IQR Method):")
print(outliers.tail())

Outliers (IQR Method):
Ticker       AAPL   GOOG   MSFT
Date                           
2025-02-05  False  False  False
2025-02-06  False  False  False
2025-02-07  False  False  False
2025-02-10  False  False  False
2025-02-11  False  False  False


In [7]:
cleaned_data = data.copy()
cleaned_data[outliers] = None
cleaned_data = cleaned_data.interpolate(method='linear')
cleaned_data

Price            Close                                High              \
Ticker            AAPL        GOOG        MSFT        AAPL        GOOG   
Date                                                                     
2025-01-13  234.142563  192.289993  417.190002  234.412271  192.490005   
2025-01-14  233.023788  191.050003  415.670013  235.860666  193.270004   
2025-01-15  237.608749  196.979996  426.309998  238.697564  197.800003   
2025-01-16  228.009308  194.410004  424.579987  237.748600  196.979996   
2025-01-17  229.727417  197.550003  429.029999  232.034878  198.809998   
2025-01-21  222.395477  199.630005  428.500000  224.173521  203.839996   
2025-01-22  223.584167  200.029999  446.200012  223.873842  202.119995   
2025-01-23  223.414368  199.580002  446.709991  226.780662  201.940002   
2025-01-24  222.535324  201.899994  444.059998  225.382200  202.570007   
2025-01-27  229.607544  193.770004  434.559998  231.895022  198.669998   
2025-01-28  237.998322  197.070007  447.200012  239.926210  197.229996   
2025-01-29  239.097122  197.179993  442.329987  239.596573  198.460007   
2025-01-30  237.329056  202.630005  414.989990  240.525538  203.238007   
2025-01-31  235.740814  205.600006  415.059998  246.918527  207.080002   
2025-02-03  227.759583  202.639999  410.920013  231.575395  205.220001   
2025-02-04  232.544327  207.710007  412.369995  232.873966  208.699997   
2025-02-05  232.214691  193.300003  413.290009  232.414468  194.550003   
2025-02-06  232.963867  193.309998  415.820007  233.543232  193.830002   
2025-02-07  227.380005  187.139999  409.750000  233.743004  193.014999   
2025-02-10  227.649994  188.199997  412.220001  230.589996  189.990005   
2025-02-11  232.619995  187.070007  411.440002  235.229996  188.800003   

Price                          Low                                Open  \
Ticker            MSFT        AAPL        GOOG        MSFT        AAPL   
Date                                                                     
2025-01-13  418.500000  229.467710  188.660004  412.290009  233.273523   
2025-01-14  419.739990  232.214681  189.639999  410.720001  234.492175   
2025-01-15  428.149994  234.172525  193.330002  418.269989  234.382301   
2025-01-16  429.489990  227.779565  194.300003  424.390015  237.089336   
2025-01-17  434.480011  228.229064  195.309998  428.170013  231.865066   
2025-01-21  430.899994  219.139063  199.440002  425.600006  223.753984   
2025-01-22  447.269989  219.548596  199.199997  436.000000  219.548596   
2025-01-23  446.750000  222.055861  196.820007  441.500000  224.493183   
2025-01-24  446.649994  221.166834  199.783997  441.399994  224.533128   
2025-01-27  435.200012  223.733997  192.699997  423.500000  223.773962   
2025-01-28  448.380005  230.556507  192.610001  431.380005  230.596471   
2025-01-29  446.880005  233.752992  195.190002  440.399994  233.862872   
2025-01-30  422.859985  236.949483  199.472000  413.160004  238.407871   
2025-01-31  420.690002  233.183628  203.580002  414.910004  246.918527   
2025-02-03  415.410004  225.452122  201.660004  408.660004  229.737419   
2025-02-04  413.920013  226.401072  204.259995  409.739990  227.000419   
2025-02-05  413.829987  228.019307  189.910004  410.399994  228.279016   
2025-02-06  418.200012  230.176923  190.490005  414.000000  231.035979   
2025-02-07  418.649994  227.010401  185.100006  408.100006  232.344548   
2025-02-10  415.459991  227.199997  187.610001  410.920013  229.570007   
2025-02-11  412.489594  228.130005  186.080002  409.299988  228.199997   

Price                                  Volume                      
Ticker            GOOG        MSFT       AAPL      GOOG      MSFT  
Date                                                               
2025-01-13  191.350006  415.239990   49630700  13169100  17604800  
2025-01-14  192.500000  417.809998   39435300  13651200  16935900  
2025-01-15  194.350006  419.130005   39832000  12894900  19637800  
2025-01-16  195.824997  428.700012   7

In [9]:
import os
sim_ticker_path =  os.path.join(os.getcwd(), "sim_tickers.txt") 
with open(sim_ticker_path) as f:
    sim_tickers = f.read().splitlines()
print(sim_tickers[0:5])



['HOOD', 'EQT', 'SW', 'CVNA', 'TPL']

In [ ]:
from requests import Session
from pyrate_limiter import Duration, Limiter, RequestRate
from requests_cache import CacheMixin, SQLiteCache # type: ignore
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket # type: ignore

class CachedLimiterSession(CacheMixin, LimiterMixin, Session): # type: ignore
    """
    A custom session class that combines caching and rate-limiting.
    This class explicitly handles the type mismatch between `requests.Session` and `CacheMixin`.
    """
    pass
session = CachedLimiterSession(
    limiter=Limiter(
        RequestRate(
            5, Duration.SECOND
        )  # Rate limit per minute
    )
)

In [20]:
sim_data =  yf.download(sim_tickers, session=session, repair=True)

[*********************100%***********************]  2466 of 2466 completed

153 Failed downloads:
['WEYS', 'BRY', 'DTC', 'MAT', 'BFAM', 'ARHS', 'LNKB', 'TSVT', 'UVSP', 'ULH', 'CMA', 'ALRM', 'ARWR', 'PLPC', 'ATRO', 'LCII', 'SONO', 'JBGS', 'PBPB', 'PRAA', 'UMH', 'TGTX', 'BCML', 'CASY', 'SLRN', 'EYE', 'NWFL', 'CERT', 'FELE', 'APEI', 'RBCAA', 'WEAV', 'FHN', 'ARQT', 'DDS', 'HOMB', 'W', 'ALMS', 'ACIC', 'MPX', 'FORM', 'PRCT', 'FSBW', 'MBIN', 'NTNX', 'OLP', 'ECG', 'CLYM', 'NTB', 'AGM', 'HWC', 'COHU', 'AMTB', 'CBRL', 'POWI', 'SG', 'TCX', 'GBCI', 'ACNB', 'CMTG', 'NEXT', 'UI', 'MTDR', 'AMWD', 'CTV', 'NBHC', 'RBC', 'SBH', 'RIOT', 'GTXI', 'OLMA', 'OII', 'FCFS', 'ESNT', 'CSTE', 'ACHV', 'PRTA', 'PD', 'BALY', 'XRAY', 'CIFR', 'CCNE', 'SPRY', 'EXEL', 'SMPL', 'REAX', 'NLY', 'CFLT', 'MC', 'PBI', 'AUB', 'HELE', 'SOFI', 'FLIC', 'SFBS', 'CTNM', 'VCSA', 'PRVA', 'GRC', 'SMBC', 'PATH', 'REI', 'PENN', 'ATSG', 'AMED', 'VMEO', 'INBX', 'CADE', 'DTI', 'FLS', 'HDSN', 'PTEN', 'JXN', 'SAGE', 'ZION', 'SITM', 'CNXN', 'MG

In [57]:

close = sim_data["Close"]
returns = close.pct_change(fill_method=None)
mean_returns = returns.mean(skipna=True)
sd_returns = returns.std(ddof=1)


In [58]:
import pandas as pd
data_to_save = pd.DataFrame(
    {"mean": mean_returns, 
     "sd": sd_returns,
     "ticker": mean_returns.index}
)
data_to_save.to_parquet("sim_ticker_stats.parquet", index=False)

In [59]:
from math import exp
import random
def simulate_returns(mean, sd, start= 100):
    shock =  random.gauss()*sd
    drift = mean - (0.5 * sd**2)
    return start + (start * (drift + shock))

def simulate_returns2(mean, sd, start= 100):
    return start * exp((mean - 0.5 * sd**2) + random.gauss() * sd)

In [60]:
def simulate_returns_from_stats(stats, timesteps, start=100):
    data = []
    for i in range(timesteps):
        data.append(simulate_returns2(stats["mean"], stats["sd"], start))
        start = data[-1]
    return data

stats = data_to_save.iloc[0]
simulated_data = simulate_returns_from_stats(stats, 100)

In [62]:
simulated_data 

[98.53588721932044,
 93.65133957212214,
 93.9121483761904,
 95.60981907094656,
 94.57756510233175,
 99.3456697506535,
 97.45373915805386,
 97.28112208094228,
 94.29475549978548,
 96.49110916751343,
 98.73720397262065,
 98.79473838588827,
 98.75398839838329,
 98.05555510801995,
 101.35758270470215,
 101.97630202995774,
 100.74685626684395,
 100.18339360426388,
 97.02864776728644,
 93.35942663517544,
 92.17765160442276,
 92.75608091626484,
 93.08847189534373,
 92.42563250108103,
 96.29320843207809,
 99.25512787333774,
 99.91089421333926,
 100.94443818297314,
 96.8338519562465,
 98.41070815155676,
 99.45280960064026,
 98.81176381596592,
 98.51696538090795,
 97.04670942751054,
 95.76789581909355,
 97.28054451008113,
 100.74017494197646,
 104.96849561925522,
 100.0730313389519,
 99.15687120227749,
 97.49610359574827,
 98.61558953527457,
 99.11871009830026,
 101.1013288136411,
 101.1794764615094,
 101.99164552083818,
 102.99691496198562,
 105.27479068312134,
 105.73348147993363,
 103.7188794